# Just Bundle Things

This script runs through some things you can do with owmeta bundles

 - [BASICS](#BASICS) Creating and listing bundles
 - [FETCH](#FETCH) Getting bundles
 - [LOCAL](#LOCAL) Local index and local cache
 - [REMOTE](#REMOTE) Remote repositories and distributed file shares
 - [INSTALL](#INSTALL) Installing a bundle to the local index

We'll do everything in a temporary directory and clean up at the end

In [5]:
working_directory=$(mktemp -d)
cd $working_directory
cleanup () {
    cd /
    if [ -d "$working_directory" ] ; then
        rm -rf $working_directory
    fi
}
# Clean up when the script exits normally or with an "INT" signal
trap cleanup EXIT INT

In [8]:
echo $(pwd)

/tmp/tmp.Z2blYPTlnH


We'll also be using the common owmeta database

In [18]:
rm -rf .owm
owm clone https://github.com/openworm/OpenWormData.git --branch=461-pow-om-rename

Cloning...
100%|████████████████████████████████| 271.0/271.0 [00:05<00:00, 45.30objects/s]
Deserializing...
0 ctx [00:00, ? ctx/s]
 78%|██████████████████████████████▌        | 158/202 [00:07<00:00, 51.22 ctx/s]
65832 triples [00:07, 2514.93 triples/s]
 81%|███████████████████████████████▋       | 164/202 [00:07<00:01, 36.45 ctx/s]
68078 triples [00:07, 3846.46 triples/s]
 99%|██████████████████████████████████████▍| 199/202 [00:08<00:00, 39.19 ctx/s]
                                                                                
Finalizing writes to database...     
100%|███████████████████████████████████████| 202/202 [00:08<00:00, 39.19 ctx/s]
                                                                                
Loaded 73,805 triples                
100%|███████████████████████████████████████| 202/202 [00:08<00:00, 39.19 ctx/s]
73805 triples [00:08, 4628.33 triples/s]
100%|███████████████████████████████████████| 202/202 [00:08<00:00, 24.03 ctx/s]
Done!


## BASICS

To create a bundle, you have to register a bundle descriptor. This is a file which describes what's in a bundle.

You can make a bundle descriptor file with any editor you like. It's a YAML serialization.

In [19]:
cat > bundle.yml << HERE
---
id: example/abundle
description: |-
    This is a bundle used for demonstration purposes
includes:
    - http://openworm.org/schema/sci
patterns:
    - rgx:.*data_sources.*#Neurons/context_for.*
    - '*/translators/*'
files:
    includes:
        - src/readme.txt
    patterns:
        - 'src/data_set_0[0-9][0-9]/**'
HERE

The `owm bundle register` command actually registers the bundle. This puts a reference to the descriptor file in the .owm directory


In [20]:
owm bundle register bundle.yml

`owm bundle list` lists the bundles registered in this owmeta project

In [21]:
owm bundle list

example/abundle - This is a bundle used for demonstration purposes


If you move or rename a bundle file owmeta will not know about it: it does not track file moves. It will, however, tell you if a bundle descriptor cannot be found when you list registered bundles.

In [22]:
mv bundle.yml aBundle.yml
owm bundle list

example/abundle - ERROR: Cannot read bundle descriptor at 'bundle.yml'


To correct this, you must re-register the bundle at the new location

In [25]:
owm bundle register aBundle.yml
owm bundle list

example/abundle - This is a bundle used for demonstration purposes


If you're done with a bundle, you deregister it. You can provide either the descriptor file name or the bundle name

In [28]:
owm bundle deregister 'example/abundle'
owm bundle list

## FETCH

[BASIC](#BASIC) showed us how to create and list bundles, but what about when you want someone else's bundles? To fetch a bundle, you need the bundle name. That name is queried for in your local repository (see below in [LOCAL](#LOCAL)), then in any remotes configured in your ".owm" directory or user settings, and finally the default remote (see [REMOTE](#REMOTE)). `owm bundle fetch` does this for you.

In [29]:
owm bundle fetch example/bundle.00

No loader could be found for "example/bundle.00"


: 1

Fetching a bundle puts it in your local repository so you can use it in any projects on the local machine. You use a bundle in Python with the Bundle object. You can access contexts within a bundle by passing the context to the bundle as shown below.

In [ ]:
use_a_bundle 'example/bundle.01' \
    'https://openworm.org/data#example_bundle_context'

Note that the bundle does not need to have been already fetched. use_a_bundle.py, we use a bundle example/bundle.01 which we had not previously fetched. When you make the Bundle object, owmeta will retrieve the bundle from remotes if necessary.